In [2]:
import pandas as pd
import numpy as np
from sklearn import tree
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from tabulate import tabulate

In [3]:
df = pd.read_csv("new_train.csv")

In [6]:
df['rank_rel'] = np.zeros(len(df))
df['rank_rel'] = np.where(df.click_bool == 1, 1, df.rank_rel)
df['rank_rel'] = np.where(df.booking_bool == 1, 5, df.rank_rel)

In [5]:
df= df.drop(['Unnamed: 0'], axis = 1)

In [7]:
sample_srch_id = np.random.choice(df.srch_id.unique(), size= int(0.1 * len(df.srch_id.unique())))
df_sample = df[df.srch_id.isin(sample_srch_id)]
print len(df_sample)
print df_sample['click_bool'].corr(df['click_bool'])
print df_sample['booking_bool'].corr(df['booking_bool'])

471048
0.9999999999999999
0.9999999999999998


In [8]:
grouped  = df_sample[df_sample.click_bool == False].groupby('srch_id')
grouped = grouped.apply(lambda x: x.sample(frac = 0.2))
grouped = grouped.apply(lambda x: x) 
df_sample = df_sample.drop(df_sample[df_sample.click_bool == False].index).append(grouped)

In [18]:
y_rank = df_sample[['rank_rel']]
X = df_sample.drop(['position','rank_rel','click_bool','booking_bool'], axis = 1)

In [20]:
X_train = X.head(2*len(df)/3) 
X_test = X.tail(len(df)/3) 
y_train = y_rank.head(2*len(df)/3) 
y_test = y_rank.tail(len(df)) 

In [22]:
gbm = GradientBoostingClassifier()
model = gbm.fit(X_train, y_train)

C:\Users\Maria\Anaconda2\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
pred2 = model.predict(X_test)

In [24]:
print gbm.score(X_test, y_test)

0.8146399912857195


In [26]:
print sum(pred2)

10166.0


In [27]:
gbm = GradientBoostingClassifier(max_depth = 2)

In [28]:
y_test2 = df_sample[['position']].loc[X_test.index]
y_train2 = df_sample[['position']].loc[X_train.index]

In [29]:
model2 = gbm.fit(X_train, y_train2)

In [30]:
pred = model2.predict(X_test)

In [31]:
print gbm.score(X_test, y_test2)

0.10370901564939544


In [32]:
print pred

[24 30  4 ...  6  7  6]


In [33]:
dfFinal = X_test

In [41]:
dfFinal['position'] = pred
dfFinal['rank_rel'] = pred2

In [42]:
dfFinal = dfFinal.sort_values(by=['srch_id'])

In [43]:
dfFinal

,srch_id,prop_country_id,year,month,prop_score,prop_location_score1,prop_location_score2,diff_hist_price,usd_diff,star_diff,...,random_bool,price_usd,total_price,nr_pers,comp2_rate,comp5_rate,comp8_rate,prop_id,position,rank_rel
63,6,100,2013,6,0.000005,1.95,0.0000,-144.000000,145.00,3.00,...,False,145.00,145.00,2,0.0,0.0,0.0,104251,24,0.0
"(6, 61)",6,100,2013,6,0.000005,1.95,0.0000,-114.030000,115.03,0.00,...,False,115.03,115.03,2,0.0,0.0,0.0,22135,30,0.0
"(12, 127)",12,158,2013,3,0.000003,3.26,0.0000,37.487509,49.00,3.00,...,True,49.00,49.00,3,0.0,0.0,0.0,47902,4,0.0
"(12, 120)",12,158,2013,3,0.000003,3.14,0.0000,42.487509,44.00,3.00,...,True,44.00,44.00,3,0.0,1.0,0.0,10881,6,0.0
"(12, 138)",12,158,2013,3,0.000004,2.64,0.0000,6.521082,36.00,3.00,...,True,36.00,36.00,3,0.0,0.0,0.0,105912,4,0.0
145,12,158,2013,3,4761.000000,0.00,0.0476,15.521082,27.00,3.00,...,True,27.00,27.00,3,0.0,0.0,0.0,129310,1,0.0
"(12, 121)",12,158,2013,3,0.000009,1.10,0.0000,31.269464,51.00,4.00,...,True,51.00,51.00,3,0.0,0.0,0.0,12510,10,0.0
"(12, 125)",12,158,2013,3,0.000003,3.30,0.0000,46.487509,40.00,3.00,...,True,40.00,40.00,3,0.0,1.0,0.0,43250,4,0.0
"(36, 347)",36,219,2013,4,0.030942,1.39,0.0430,-31.734630,199.07,4.00,...,False,199.07,199.07,1,0.0,0.0,0.0,18970,6,0.0
"(36, 368)",36,219,2013,4,0.004507,0.69,0.0031,50.913159,97.50,3.00,...,False,97.50,97.50,1,0.0,0.0,0.0,133389,1,0.0


In [44]:
df = dfFinal[['srch_id', 'position', 'rank_rel']]



def dcg(i, rel):

    return (2**rel - 1)/(np.log2(i + 1))



#########################################################################################



# CACULATING DCG



sum_dcg = 0

array_dcg = []



for k in range(0, len(df) - 1):

    if k != len(df) - 1:
        
        if(df['srch_id'].iloc[k] < df['srch_id'].iloc[k + 1]):
 

            sum_dcg += dcg(df['position'].iloc[k],df['rank_rel'].iloc[k])

            array_dcg.append(sum_dcg)
            sum_dcg = 0

        else:

            sum_dcg += dcg(df['position'].iloc[k],df['rank_rel'].iloc[k])

    else:

        sum_dcg += dcg(df['position'].iloc[k],df['rank_rel'].iloc[k])

        array_dcg.append(sum_dcg)

        sum_dcg = 0

# CALCULATING IDCG
array_idcg = np.array([])
t_idcg = np.array([])
sum_idcg = 0

for k in range(0, len(df) - 1):
    if k != len(df) - 1:
        if(df['srch_id'].iloc[k] < df['srch_id'].iloc[k + 1]):
            t_idcg = np.append(t_idcg, df['rank_rel'].iloc[k])
            t_idcg = np.sort(t_idcg)[::-1]
            for j in range(0, len(t_idcg)):
                sum_idcg += dcg(j + 1, t_idcg[j])
                if(sum_idcg > 0):
                    print sum_idcg
            array_idcg = np.append(array_idcg, sum_idcg)
            sum_idcg = 0
            t_idcg = []
        else:
            t_idcg = np.append(t_idcg, df['rank_rel'].iloc[k])   
    else:
        t_idcg = np.append(t_idcg, df['rank_rel'].iloc[k])
        t_idcg = np.sort(t_idcg)[::-1]
        for j in range(0, len(t_idcg)):
            sum_idcg += dcg(j + 1, t_idcg[j])
        array_idcg = np.append(array_idcg, sum_idcg)
        
print 'nDGC = ', (np.sum(array_dcg)/np.sum(array_idcg))


31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
50.55882236071518
50.55882236071518
31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
31.0
31.630929753571458
31.630929753571458
31.630929753571458
31.630929753571458
31.630929753571458
31.630929753571458
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31

31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.630929753571458
31.630929753571458
31.630929753571458
31.630929753571458
31.630929753571458
31.630929753571458
31.0
50.55882236071518
66.05882236071518
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
3

31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.6309297535714575
1.6309297535714575
1.0
1.6309297535714575
2.1309297535714578
2.5616063116448506
2.5616063116448506
2.5616063116448506
2.5616063116448506
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
50.55882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
3

1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
50.55882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.630929753571458
31.630929753571458
31.630929753571458
31.630929753571458
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
50.55882236071518
51.05882236071518
51.05882236071518
51.05882236071518
51.05882236071518
51.05882236071518
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
31.0
31.0

31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
31.0
50.55882236071518
50.55882236071518
31.0
50.55882236071518
50.55882236071518
50.55882236071518
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.6309

31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.630929753571458
31.630929753571458
31.630929753571458
31.630929753571458
31.630929753571458
31.630929753571458
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
50.55882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
50.55882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
3

31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
50.55882236071518
51.05882236071518
51.05882236071518
51.05882236071518
51.05882236071518
51.05882236071518
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0

31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0

31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.

31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
31.0
50.55882236071518
50.55882236071518
50.55882236071518
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.